<a href="https://colab.research.google.com/github/mk7exe/millerite/blob/master/Phase3/Millerite_NN_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import h5py
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import datetime

import matplotlib.pyplot as plt
%matplotlib inline


Loading the dataset to evaluate from Github.

In [27]:
! wget -nv https://github.com/mk7exe/millerite/raw/master/Phase2/datasets/surf_dataset.h5
root_path = '.'
dataset = h5py.File(root_path + '/surf_dataset.h5', "r")
x_orig = np.array(dataset["test_set_x"][:]).T 
y_orig = np.array(dataset["test_set_y"][:]) 

2020-04-05 20:51:37 URL:https://raw.githubusercontent.com/mk7exe/millerite/master/Phase2/datasets/surf_dataset.h5 [6464/6464] -> "surf_dataset.h5.2" [1]


Mounting google drive to load trained model.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Defining the model to load (n case there are moultiple models saved in Google Drive)

In [0]:
drive_path = 'gdrive/My Drive/Colab Notebooks/models'
model_date = '20200405-013816'
model_path = drive_path + '/' + model_name

Getting feature scalling parameters from Google Drive

In [37]:
xmean_path = model_path + '/xmean.txt'
xmean = np.genfromtxt(xmean_path, delimiter=',')

xrange_path = model_path + '/xrange.txt'
xrange = np.genfromtxt(xrange_path, delimiter=',')

y_path = model_path + '/ymean_yrange.txt'
with open(y_path) as f:
  lines=f.readlines()
  temp = line.split()
  ymean = temp[0]
  yrange = temp[1]

OSError: ignored

Applying the feature scaling to x

In [0]:
x = np.divide(np.subtract(x_orig, xmean), xrange)

Defining the model

In [0]:
hparams = {
    'num_layers': 3,
    'num_units': 50,
    'dropout': 0.1,
    'alpha': 0.0001,
    'lambdaL2': 0.01,
    'last_activation': 'tanh',
    'activation': 'relu'
    }

def train_model(hparams):
  num_layers = hparams['num_layers']
  num_units = hparams['num_units']
  activation = hparams['activation']
  lambdaL2 = hparams['lambdaL2']
  dropout = hparams['dropout']
  learning_rate = hparams['alpha']  
  out_activation = hparams['last_activation']

  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=x.shape[1:]))

  for i in range(num_layers):
    model.add(keras.layers.Dense(num_units, kernel_regularizer=
                               keras.regularizers.l2(lambdaL2), use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation(activation))
    model.add(keras.layers.Dropout(dropout))

  model.add(keras.layers.Dense(1, activation=out_activation))

  adam = keras.optimizers.Adam(learning_rate=learning_rate,
                               beta_1=0.9, beta_2=0.999) 
  model.compile(optimizer=adam, 
                loss=keras.losses.mean_squared_error, 
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
  return model

Loading the model

In [44]:
model = train_model(hparams)
model.load_weights(model_path)

In [53]:
y_pred = model.predict(x)
yrange = float(yrange)
ymean = float(ymean)

y_bar = np.multiply(y_pred, yrange) + ymean 

for i in range(len(y_pred)):
  print(y_bar[i], y_orig[i], np.divide(np.abs(y_bar[i])-np.abs(y_orig[i]), 
        y_orig[i]))

[32.367893] 18.64505600000001 [0.7360041]
[42.8839] 47.06206799999973 [-0.08877997]
[68.49482] 76.20236799999975 [-0.1011458]
[64.38195] 115.18256799999972 [-0.44104433]
[86.11937] 74.39596799999981 [0.15758118]
[36.121037] 21.360388 [0.6910291]
[32.523674] 18.791684000000032 [0.7307483]
[35.48716] 37.057783999999856 [-0.04238314]
